In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import json

In [2]:
def get_regional_csvs(region: str):
    """
    Returns a list of dataframes of by region
    """
    dfs = {}
    type_dict = {'Champion1': 'int16', 'Champion2': 'int16',
             'Champion3': 'int16', 'Champion4': 'int16',
             'Champion5': 'int16'}
    for f in os.listdir('data'):
        if region in f:
            dfs[f.split('_')[1].split('.')[0]] = pd.read_csv(f'data/{f}', dtype=type_dict)
    return dfs

In [3]:
def get_top3_dfs(region: str):
    """
    Returns at list of dataframes of by region of the top 3 champions per lane per player.
    """
    dfs = {}
    for f in os.listdir('data/top3'):
        if region in f:
            dfs[f.split('_')[1].split('.')[0]] = pd.read_csv(f'data/top3/{f}')
    return dfs

### Load in the Champions by id to map

In [4]:
raw_champions = None
with open('data/champions.json', encoding="cp866") as f:
    raw_champions = json.load(f)
    
champion_ids_name = {}
for champ in raw_champions['data'].keys():
    champion_ids_name[int(raw_champions['data'][champ]['key'])] = champ

### Combine the regional datasets into one mega dataset

In [5]:
na_top3 = get_top3_dfs('na')
na_top3['diamond'].head()

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
0,51,272295,bot,145,88254,bot,103,74100,mid,84,...,jg,99,9746,sup,254,7431,jg,114,1286,top
1,55,67222,mid,238,51605,mid,3,51363,mid,39,...,sup,16,10862,sup,202,9299,bot,21,8998,bot
2,39,956886,top,90,109817,mid,98,74811,top,245,...,mid,350,16135,sup,1,14686,mid,18,13146,bot
3,64,154967,jg,517,97859,mid,238,66095,mid,104,...,bot,145,17146,bot,497,6812,sup,555,6105,sup
4,131,514349,jg,103,214265,mid,99,163929,sup,25,...,mid,38,10785,mid,11,9387,jg,113,6642,jg


In [7]:
euw_top3 = get_top3_dfs('euw')
euw_top3['diamond'].head()

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
0,30,1115282,jg,9,93596,jg,72,65422,jg,119,...,bot,99,12783,sup,555,11723,sup,267,7108,sup
1,202,185698,bot,110,146405,bot,51,142765,bot,157,...,sup,517,46720,mid,412,45436,sup,40,38184,sup
2,432,91600,sup,64,35730,jg,412,35321,sup,111,...,bot,76,16636,jg,517,14381,mid,67,14038,bot
3,84,743488,top,246,83962,mid,92,49463,top,64,...,bot,497,6136,sup,44,4312,sup,43,4260,sup
4,145,112281,bot,15,92945,bot,29,54843,bot,101,...,top,157,9270,mid,35,9021,jg,38,5469,mid


In [8]:
kr_top3 = get_top3_dfs('kr')
kr_top3['diamond'].head()

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
0,64,66850,jg,104,45310,jg,76,35127,jg,4,...,bot,101,6751,sup,89,4349,sup,350,3814,sup
1,412,79442,sup,236,73300,bot,3,72863,mid,51,...,jg,59,17225,jg,266,14354,top,54,12243,top
2,523,231271,bot,81,135222,bot,145,129195,bot,57,...,top,157,11361,mid,107,7378,top,1,6774,mid
3,92,363189,top,238,223867,mid,39,193324,top,268,...,jg,117,11290,sup,26,10342,sup,67,8515,bot
4,421,1697105,jg,412,706123,sup,64,218576,jg,157,...,sup,81,36247,bot,236,32172,bot,67,25542,bot


In [13]:
na_top3_combined = pd.concat(na_top3.values(), axis=0)
na_top3_combined.shape

(3665, 45)

In [12]:
euw_top3_combined = pd.concat(euw_top3.values(), axis=0)
euw_top3_combined.shape

(11915, 45)

In [11]:
kr_top3_combined = pd.concat(kr_top3.values(), axis=0)
kr_top3_combined.shape

(7992, 45)

In [14]:
all_combined = pd.concat([na_top3_combined, euw_top3_combined, kr_top3_combined], axis=0)
all_combined.shape

(23572, 45)

In [15]:
all_combined.head()

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
0,11,228838,jg,141,127298,jg,107,75772,top,555,...,sup,12,26501,sup,84,21316,top,51,12740,bot
1,99,158190,sup,16,108973,sup,267,101700,sup,21,...,jg,163,17509,jg,39,16153,top,10,11956,top
2,84,134256,top,145,124395,bot,157,87764,mid,99,...,sup,245,27580,jg,254,25642,jg,875,24216,top
3,267,107035,sup,350,99720,sup,16,46863,sup,68,...,mid,81,9618,bot,202,8821,bot,523,5726,bot
4,21,131246,bot,75,74165,top,54,67741,top,17,...,mid,32,10750,sup,90,9949,mid,45,8198,mid


### Replace champion Ids with Champion names

In [16]:
all_combined = all_combined.replace(champion_ids_name)
all_combined.head()

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
0,MasterYi,228838,jg,Kayn,127298,jg,Rengar,75772,top,Pyke,...,sup,Alistar,26501,sup,Akali,21316,top,Caitlyn,12740,bot
1,Lux,158190,sup,Soraka,108973,sup,Nami,101700,sup,MissFortune,...,jg,Taliyah,17509,jg,Irelia,16153,top,Kayle,11956,top
2,Akali,134256,top,Kaisa,124395,bot,Yasuo,87764,mid,Lux,...,sup,Ekko,27580,jg,Vi,25642,jg,Sett,24216,top
3,Nami,107035,sup,Yuumi,99720,sup,Soraka,46863,sup,Rumble,...,mid,Ezreal,9618,bot,Jhin,8821,bot,Aphelios,5726,bot
4,MissFortune,131246,bot,Nasus,74165,top,Malphite,67741,top,Teemo,...,mid,Amumu,10750,sup,Malzahar,9949,mid,Veigar,8198,mid


### Set dtypes to correct type

In [28]:
all_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23572 entries, 0 to 1525
Data columns (total 45 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Champion1   23572 non-null  object
 1   Mastery1    23572 non-null  int64 
 2   Lane1       23572 non-null  object
 3   Champion2   23572 non-null  object
 4   Mastery2    23572 non-null  int64 
 5   Lane2       23572 non-null  object
 6   Champion3   23572 non-null  object
 7   Mastery3    23572 non-null  int64 
 8   Lane3       23572 non-null  object
 9   Champion4   23572 non-null  object
 10  Mastery4    23572 non-null  int64 
 11  Lane4       23572 non-null  object
 12  Champion5   23572 non-null  object
 13  Mastery5    23572 non-null  int64 
 14  Lane5       23572 non-null  object
 15  Champion6   23572 non-null  object
 16  Mastery6    23572 non-null  int64 
 17  Lane6       23572 non-null  object
 18  Champion7   23572 non-null  object
 19  Mastery7    23572 non-null  int64 
 20  Lane7  

In [38]:
all_combined[all_combined['Mastery13'] == 'Sett']

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
361,Janna,45283,sup,Yuumi,42723,sup,Trundle,30631,jg,Udyr,...,mid,Sivir,Sett,bot,Zoe,843,mid,Varus,452,bot


In [40]:
all_combined['Mastery13'][all_combined['Mastery15'].apply(lambda x: isinstance(x, str))]

178     2669
527     3672
173     6776
148     2954
373     1623
661     3453
2006    4083
2100    2681
186     3124
340      967
687     6195
202     3559
688     Gnar
17      Nami
1203     377
109     3285
247     2746
373     1093
1190    6221
1388     851
1416     309
113     4166
155     Yone
273      845
1090    2104
Name: Mastery13, dtype: object

In [41]:
def is_valid_mastery(x):
    if isinstance(x, str):
        if x.isnumeric():
            return True
        else:
            return False
    return True

In [46]:
# Mastery 13, 14, and 15 are the only affected columns
valid_rows = all_combined['Mastery13'].apply(is_valid_mastery) & all_combined['Mastery14'].apply(is_valid_mastery) & all_combined['Mastery15'].apply(is_valid_mastery)

In [44]:
all_combined.shape

(23572, 45)

In [48]:
all_combined = all_combined[valid_rows]
all_combined.shape

(23541, 45)

In [49]:
all_combined = all_combined.astype({f'Mastery{i}': np.int64 for i in range(1, 16)})

In [50]:
all_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23541 entries, 0 to 1525
Data columns (total 45 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Champion1   23541 non-null  object
 1   Mastery1    23541 non-null  int64 
 2   Lane1       23541 non-null  object
 3   Champion2   23541 non-null  object
 4   Mastery2    23541 non-null  int64 
 5   Lane2       23541 non-null  object
 6   Champion3   23541 non-null  object
 7   Mastery3    23541 non-null  int64 
 8   Lane3       23541 non-null  object
 9   Champion4   23541 non-null  object
 10  Mastery4    23541 non-null  int64 
 11  Lane4       23541 non-null  object
 12  Champion5   23541 non-null  object
 13  Mastery5    23541 non-null  int64 
 14  Lane5       23541 non-null  object
 15  Champion6   23541 non-null  object
 16  Mastery6    23541 non-null  int64 
 17  Lane6       23541 non-null  object
 18  Champion7   23541 non-null  object
 19  Mastery7    23541 non-null  int64 
 20  Lane7  

### Helper function that gets the top 3 champions by lane for someone who mains a champion

In [51]:
def get_top3_by_champion(champ: str, df: pd.DataFrame):
    champ_df = df[df['Champion1'] == champ].copy()
    if champ_df.shape[0] == 0:
        return "Not enough data!"
    champ_df[[f'Mastery{i}' for i in range(2, 16)]] = (1-champ_df[[f'Mastery{i}' for i in range(2, 16)]].div(champ_df[[f'Mastery{i}' for i in range(2, 16)]][[f'Mastery{i}' for i in range(2, 16)]].sum(axis=1), axis=0))
    champ_lanes = champ_df[[f'Lane{i}' for i in range(2, 16)]].stack().reset_index(drop=True)
    champ_mastery = champ_df[[f'Mastery{i}' for i in range(2, 16)]].stack().reset_index(drop=True)
    champ_champs = champ_df[[f'Champion{i}' for i in range(2, 16)]].stack().reset_index(drop=True)
    champ_combined = pd.concat([champ_lanes, champ_mastery, champ_champs], axis=1)
    champ_combined.columns = ['Lane', 'Mastery', 'Champion']
    lanes = ['top', 'jg', 'mid', 'bot', 'sup']
    dfs = pd.DataFrame(columns=['Champion', 'Mastery', 'Lane'])
    for lane in lanes:
        counts = champ_combined.groupby(['Lane', 'Champion']).count().loc[lane, :].sort_index()
        means = champ_combined.groupby(['Lane', 'Champion']).mean().loc[lane, :].sort_index()
        lane_champs = means.div(counts, axis=1).sort_values('Mastery', ascending=True).iloc[:3].reset_index()
        lane_champs['Lane'] = [lane] * 3
        dfs = pd.concat([dfs, lane_champs])
    return dfs

### Get the top 3 champions per lane per champion

In [59]:
champ_dfs = {}
for champ in champion_ids_name.values():
    response = get_top3_by_champion(champ, all_combined)
    if isinstance(response, str):
        print(champ)
        print("Not enough data for that champ")
    champ_dfs[champ] = response

Renata
Not enough data for that champ


In [60]:
champ_dfs['Ziggs']

,Champion,Mastery,Lane
0,Teemo,0.134465,top
1,Garen,0.157856,top
2,Malphite,0.186343,top
0,Nunu,0.115756,jg
1,Shaco,0.149309,jg
2,Diana,0.152086,jg
0,Ahri,0.130632,mid
1,Veigar,0.132669,mid
2,Galio,0.134642,mid
0,Ezreal,0.102959,bot


### Load the Champion to Lane Data

In [90]:
def get_champ_lane(champ: str):
    return all_combined[all_combined['Champion1'] == champ]['Lane1'].iloc[0]

### Transform the dataframes from 15 x 3 to a single row with the champion name

In [97]:
def get_final_top3_columns():
    cols = ['lane']
    lanes = ['top', 'jg', 'mid', 'bot', 'sup']
    for lane in lanes:
        for i in range(1, 4):
            cols.append(f'{lane}{i}')
            cols.append(f'{lane}_dist{i}')
    return cols

In [80]:
champ_dfs['Ziggs'].values.flatten()

array(['Teemo', 0.13446493565331413, 'top', 'Garen', 0.15785618870495352,
       'top', 'Malphite', 0.18634349780182127, 'top', 'Nunu',
       0.1157559688774835, 'jg', 'Shaco', 0.14930851440989376, 'jg',
       'Diana', 0.15208596496867682, 'jg', 'Ahri', 0.130632393327021,
       'mid', 'Veigar', 0.1326688600831376, 'mid', 'Galio',
       0.13464219570261196, 'mid', 'Ezreal', 0.10295924851228393, 'bot',
       'MissFortune', 0.11524578105975647, 'bot', 'Ashe',
       0.1301475687287968, 'bot', 'Lux', 0.07481189532742837, 'sup',
       'Morgana', 0.10338418521864229, 'sup', 'Swain',
       0.11827823536517224, 'sup'], dtype=object)

In [99]:
final_top3_data = []
final_top3_columns = get_final_top3_columns()
indicies = []
for champ in champ_dfs:
    if isinstance(champ_dfs[champ], str):
        continue
    indicies.append(champ)
    col_data = champ_dfs[champ][['Champion', 'Mastery']].values.flatten()
    col_data = np.insert(col_data, 0, get_champ_lane(champ))
    final_top3_data.append(col_data)

In [100]:
final_top3_df = pd.DataFrame(data=final_top3_data, columns=final_top3_columns, index=indicies)
final_top3_df.head()

,lane,top1,top_dist1,top2,top_dist2,top3,top_dist3,jg1,jg_dist1,jg2,...,bot2,bot_dist2,bot3,bot_dist3,sup1,sup_dist1,sup2,sup_dist2,sup3,sup_dist3
Aatrox,top,Darius,0.027552,Sett,0.028557,Camille,0.032823,LeeSin,0.019081,Kayn,...,Lucian,0.015833,Jhin,0.016731,Pantheon,0.014142,Thresh,0.019440,Pyke,0.020830
Ahri,mid,Akali,0.011050,Irelia,0.013226,Teemo,0.023614,LeeSin,0.017064,Diana,...,MissFortune,0.013629,Kaisa,0.015434,Lux,0.007601,Morgana,0.019665,Thresh,0.019927
Akali,top,Irelia,0.006435,Sett,0.019889,Camille,0.027272,LeeSin,0.008046,Ekko,...,Kaisa,0.008008,Lucian,0.009041,Pyke,0.007671,Pantheon,0.008551,Lux,0.009106
Akshan,mid,Gwen,0.104418,Sett,0.128321,Irelia,0.183478,Viego,0.114962,Graves,...,Vayne,0.132648,Lucian,0.132888,Senna,0.114934,Thresh,0.150076,Morgana,0.153998
Alistar,sup,Shen,0.048111,Malphite,0.049136,TahmKench,0.054097,LeeSin,0.033276,MasterYi,...,Ashe,0.041034,Tristana,0.042754,Thresh,0.030237,Nautilus,0.055322,Rakan,0.062517


In [101]:
final_top3_df.isna().sum().sum()

0

In [79]:
final_top3_df.to_csv('data/champion_top3_by_lane.csv')

In [181]:
na_top3_combined = na_top3_combined.replace(champion_ids_name)
na_top3_combined.head()

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
0,MasterYi,228838,jg,Kayn,127298,jg,Rengar,75772,top,Pyke,...,sup,Alistar,26501,sup,Akali,21316,top,Caitlyn,12740,bot
1,Lux,158190,sup,Soraka,108973,sup,Nami,101700,sup,MissFortune,...,jg,Taliyah,17509,jg,Irelia,16153,top,Kayle,11956,top
2,Akali,134256,top,Kaisa,124395,bot,Yasuo,87764,mid,Lux,...,sup,Ekko,27580,jg,Vi,25642,jg,Sett,24216,top
3,Nami,107035,sup,Yuumi,99720,sup,Soraka,46863,sup,Rumble,...,mid,Ezreal,9618,bot,Jhin,8821,bot,Aphelios,5726,bot
4,MissFortune,131246,bot,Nasus,74165,top,Malphite,67741,top,Teemo,...,mid,Amumu,10750,sup,Malzahar,9949,mid,Veigar,8198,mid


In [196]:
aatrox_df = na_top3_combined[na_top3_combined['Champion1'] == 'Aatrox'].copy()
aatrox_df.head()

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
244,Aatrox,526783,top,Ornn,151745,top,Rakan,103861,sup,Swain,...,bot,Ashe,15528,bot,Jinx,14787,bot,Galio,6117,mid
452,Aatrox,79163,top,Urgot,79057,top,Sylas,60301,mid,Darius,...,bot,AurelionSol,33484,mid,Lucian,23713,bot,Pantheon,23593,sup
456,Aatrox,46933,top,Lucian,40977,bot,Twitch,39645,bot,Gnar,...,mid,Cassiopeia,15003,mid,Pantheon,9262,sup,Senna,9018,sup
55,Aatrox,182187,top,Irelia,97038,top,Kaisa,73832,bot,Yasuo,...,jg,Thresh,34307,sup,Pyke,18560,sup,Senna,12755,sup
236,Aatrox,145401,top,Irelia,118386,top,Morgana,95141,sup,Nautilus,...,bot,Vladimir,32074,mid,XinZhao,28331,jg,MonkeyKing,16918,jg


In [184]:
aatrox_df[[f'Mastery{i}' for i in range(2, 16)]].sum(axis=1).head()

244    668819.0
452    641081.0
456    317929.0
55     667168.0
236    857529.0
dtype: float64

In [185]:
aatrox_df[[f'Mastery{i}' for i in range(2, 16)]].head()

,Mastery2,Mastery3,Mastery4,Mastery5,Mastery6,Mastery7,Mastery8,Mastery9,Mastery10,Mastery11,Mastery12,Mastery13,Mastery14,Mastery15
244,151745,103861,67377,59648,58066,56977,44950,26467,25184,21131,16981,15528,14787,6117
452,79057,60301,58471,56293,49645,48863,47187,44728,39958,39335,36453,33484,23713,23593
456,40977,39645,37340,29545,28863,22777,20198,17652,16991,15467,15191,15003,9262,9018
55,97038,73832,70342,57201,52014,48455,45686,43545,40008,38999,34426,34307,18560,12755
236,118386,95141,94132,87629,81971,73384,61286,53690,42305,38039,34243,32074,28331,16918


In [197]:
(1 - aatrox_df[[f'Mastery{i}' for i in range(2, 16)]].div(aatrox_df[[f'Mastery{i}' for i in range(2, 16)]].sum(axis=1), axis=0)).head()

,Mastery2,Mastery3,Mastery4,Mastery5,Mastery6,Mastery7,Mastery8,Mastery9,Mastery10,Mastery11,Mastery12,Mastery13,Mastery14,Mastery15
244,0.773115,0.844710,0.899260,0.910816,0.913181,0.914810,0.932792,0.960427,0.962346,0.968406,0.974610,0.976783,0.977891,0.990854
452,0.876682,0.905939,0.908793,0.912191,0.922560,0.923780,0.926395,0.930230,0.937671,0.938643,0.943138,0.947769,0.963011,0.963198
456,0.871113,0.875302,0.882552,0.907070,0.909216,0.928358,0.936470,0.944478,0.946557,0.951351,0.952219,0.952810,0.970868,0.971635
55,0.854552,0.889335,0.894566,0.914263,0.922038,0.927372,0.931522,0.934732,0.940033,0.941545,0.948400,0.948578,0.972181,0.980882
236,0.861945,0.889052,0.890229,0.897812,0.904410,0.914424,0.928532,0.937390,0.950666,0.955641,0.960068,0.962597,0.966962,0.980271


In [294]:
def get_top3_by_champion(champ: str):
    champ_df = na_top3_combined[na_top3_combined['Champion1'] == champ].copy()
    if champ_df.shape[0] == 0:
        return "Not enough data!"
    champ_df[[f'Mastery{i}' for i in range(2, 16)]] = (1-champ_df[[f'Mastery{i}' for i in range(2, 16)]].div(champ_df[[f'Mastery{i}' for i in range(2, 16)]][[f'Mastery{i}' for i in range(2, 16)]].sum(axis=1), axis=0))
    champ_lanes = champ_df[[f'Lane{i}' for i in range(2, 16)]].stack().reset_index(drop=True)
    champ_mastery = champ_df[[f'Mastery{i}' for i in range(2, 16)]].stack().reset_index(drop=True)
    champ_champs = champ_df[[f'Champion{i}' for i in range(2, 16)]].stack().reset_index(drop=True)
    champ_combined = pd.concat([champ_lanes, champ_mastery, champ_champs], axis=1)
    champ_combined.columns = ['Lane', 'Mastery', 'Champion']
    lanes = ['top', 'jg', 'mid', 'bot', 'sup']
    dfs = pd.DataFrame(columns=['Champion', 'Mastery', 'Lane'])
    for lane in lanes:
        counts = champ_combined.groupby(['Lane', 'Champion']).count().loc[lane, :].sort_index()
        means = champ_combined.groupby(['Lane', 'Champion']).mean().loc[lane, :].sort_index()
        lane_champs = means.div(counts, axis=1).sort_values('Mastery', ascending=True).iloc[:3].reset_index()
        lane_champs['Lane'] = [lane] * 3
        dfs = pd.concat([dfs, lane_champs])
    return dfs

In [305]:
get_top3_by_champion('Bard')

,Champion,Mastery,Lane
0,Sett,0.132452,top
1,TahmKench,0.134287,top
2,Gragas,0.235091,top
0,Ekko,0.118468,jg
1,Fiddlesticks,0.191379,jg
2,LeeSin,0.234642,jg
0,Ryze,0.193197,mid
1,Veigar,0.228261,mid
2,TwistedFate,0.241613,mid
0,Ezreal,0.117942,bot


In [199]:
aatrox_lanes = aatrox_df[[f'Lane{i}' for i in range(2, 16)]].stack().reset_index(drop=True)
aatrox_lanes.head()

0    top
1    sup
2    sup
3    sup
4     jg
dtype: object

In [204]:
aatrox_mastery = (1 - aatrox_df[[f'Mastery{i}' for i in range(2, 16)]].div(aatrox_df[[f'Mastery{i}' for i in range(2, 16)]].sum(axis=1), axis=0))

In [205]:
aatrox_mastery = aatrox_mastery.stack().reset_index(drop=True)
aatrox_mastery.head()

0    0.773115
1     0.84471
2     0.89926
3    0.910816
4    0.913181
dtype: object

In [202]:
aatrox_mastery = aatrox_df[[f'Mastery{i}' for i in range(2, 16)]].stack().reset_index(drop=True)
aatrox_mastery.head()

0    151745
1    103861
2     67377
3     59648
4     58066
dtype: object

In [206]:
aatrox_champs = aatrox_df[[f'Champion{i}' for i in range(2, 16)]].stack().reset_index(drop=True)
aatrox_champs.head()

0     Ornn
1    Rakan
2    Swain
3    Brand
4     Olaf
dtype: object

In [207]:
aatrox_combined = pd.concat([aatrox_lanes, aatrox_mastery, aatrox_champs], axis=1)
aatrox_combined.columns = ['Lane', 'Mastery', 'Champion']
aatrox_combined.head()

,Lane,Mastery,Champion
0,top,0.773115,Ornn
1,sup,0.84471,Rakan
2,sup,0.89926,Swain
3,sup,0.910816,Brand
4,jg,0.913181,Olaf


In [265]:
dfs = pd.DataFrame(columns=['Champion', 'Mastery'])

In [266]:
aatrox_test = aatrox_combined.groupby(['Lane', 'Champion']).mean().loc['mid', :].sort_index().div(aatrox_combined.groupby(['Lane', 'Champion']).count().loc['mid', :].sort_index(), axis=1).sort_values('Mastery', ascending=True).iloc[:3].reset_index()
aatrox_test

,Champion,Mastery
0,Yasuo,0.051004
1,Sylas,0.057026
2,Galio,0.094812


In [267]:
lanes_dummy = pd.Series(['top']*3, name='Lane')

In [268]:
dfs = pd.concat([dfs, aatrox_test]).join(lanes_dummy)
dfs

,Champion,Mastery,Lane
0,Yasuo,0.051004,top
1,Sylas,0.057026,top
2,Galio,0.094812,top


In [209]:
aatrox_combined.groupby(['Lane', 'Champion']).count().loc['top', :].sort_values(by='Mastery', ascending=False)

,Mastery
Champion,
Darius,10
Sett,9
Mordekaiser,9
Renekton,6
Camille,6
Irelia,5
Jax,4
Fiora,4
Garen,2


In [128]:
aatrox_combined.groupby(['Lane', 'Champion']).mean().sort_values('Mastery', ascending=False).loc['top', :]

,Mastery
Champion,
Gangplank,187246.000000
Jayce,176797.500000
Yorick,163810.000000
Ornn,151745.000000
Shen,148813.000000
Camille,140173.000000
Sion,138537.500000
Kled,137792.000000
Irelia,137179.400000


In [73]:
na_top3_combined[[f'Champion{i}' for i in range(2, 16)]].stack().reset_index(drop=True)

0        141
1        107
2        555
3        157
4        142
        ... 
51305    360
51306    142
51307     81
51308     10
51309    164
Length: 51310, dtype: int64

In [66]:
na_top3_combined.shape

(3665, 45)

In [13]:
na_top3_combined = na_top3_combined.replace(champion_ids_name)

In [14]:
na_top3_combined.head()

,Champion1,Mastery1,Lane1,Champion2,Mastery2,Lane2,Champion3,Mastery3,Lane3,Champion4,...,Lane12,Champion13,Mastery13,Lane13,Champion14,Mastery14,Lane14,Champion15,Mastery15,Lane15
0,MasterYi,228838,jg,Kayn,127298,jg,Rengar,75772,top,Pyke,...,sup,Alistar,26501,sup,Akali,21316,top,Caitlyn,12740,bot
1,Lux,158190,sup,Soraka,108973,sup,Nami,101700,sup,MissFortune,...,jg,Taliyah,17509,jg,Irelia,16153,top,Kayle,11956,top
2,Akali,134256,top,Kaisa,124395,bot,Yasuo,87764,mid,Lux,...,sup,Ekko,27580,jg,Vi,25642,jg,Sett,24216,top
3,Nami,107035,sup,Yuumi,99720,sup,Soraka,46863,sup,Rumble,...,mid,Ezreal,9618,bot,Jhin,8821,bot,Aphelios,5726,bot
4,MissFortune,131246,bot,Nasus,74165,top,Malphite,67741,top,Teemo,...,mid,Amumu,10750,sup,Malzahar,9949,mid,Veigar,8198,mid


In [21]:
na_top3_combined.groupby(['Champion1', 'Lane2'])[['Champion2']].count()

Champion2
Champion1 Lane2           
Aatrox    bot            3
          jg             3
          mid            4
          sup            3
          top           28
...                    ...
Zoe       top            6
Zyra      bot            1
          jg             2
          sup            9
          top            1

[618 rows x 1 columns]

In [85]:
na_top3_combined[na_top3_combined['Champion1'] == 'Aatrox'][['Champion2', 'Mastery2']].groupby('Champion2').sum().sort_values('Mastery2', ascending=False)

,Mastery2
Champion2,
Kayn,562731
Mordekaiser,543204
Renekton,452821
Yasuo,436275
Sylas,429179
Sett,383784
Irelia,347417
Amumu,334748
Jayce,330000


In [87]:
na_top3_combined[na_top3_combined['Champion1'] == 'Aatrox'][['Champion3', 'Mastery3']].groupby('Champion3').sum().sort_values('Mastery3', ascending=False)

,Mastery3
Champion3,
Darius,525135
Camille,401388
Pyke,384349
Sylas,347116
Rakan,283469
Yasuo,244348
Nasus,241029
Talon,240547
Fiora,208416


In [66]:
values_by_champion

{'Aatrox': [0, 0],
 'Ahri': [0, 0],
 'Akali': [0, 0],
 'Akshan': [0, 0],
 'Alistar': [0, 0],
 'Amumu': [0, 0],
 'Anivia': [0, 0],
 'Annie': [0, 0],
 'Aphelios': [0, 0],
 'Ashe': [0, 0],
 'AurelionSol': [0, 0],
 'Azir': [0, 0],
 'Bard': [0, 0],
 'Blitzcrank': [0, 0],
 'Brand': [0, 0],
 'Braum': [0, 0],
 'Caitlyn': [0, 0],
 'Camille': [0, 0],
 'Cassiopeia': [0, 0],
 'Chogath': [0, 0],
 'Corki': [0, 0],
 'Darius': [0, 0],
 'Diana': [0, 0],
 'Draven': [0, 0],
 'DrMundo': [0, 0],
 'Ekko': [0, 0],
 'Elise': [0, 0],
 'Evelynn': [0, 0],
 'Ezreal': [0, 0],
 'Fiddlesticks': [0, 0],
 'Fiora': [0, 0],
 'Fizz': [0, 0],
 'Galio': [0, 0],
 'Gangplank': [0, 0],
 'Garen': [0, 0],
 'Gnar': [0, 0],
 'Gragas': [0, 0],
 'Graves': [0, 0],
 'Gwen': [0, 0],
 'Hecarim': [0, 0],
 'Heimerdinger': [0, 0],
 'Illaoi': [0, 0],
 'Irelia': [0, 0],
 'Ivern': [0, 0],
 'Janna': [0, 0],
 'JarvanIV': [0, 0],
 'Jax': [0, 0],
 'Jayce': [0, 0],
 'Jhin': [0, 0],
 'Jinx': [0, 0],
 'Kaisa': [0, 0],
 'Kalista': [0, 0],
 'Karma': 

In [52]:
na_top3_combined[na_top3_combined['Champion1'] == 'Aatrox'][[f'Champion{i}' for i in range(2, 16)]].T.stack().reset_index().set_index('level_1').drop('level_0', axis=1)

,0
level_1,
244,Ornn
452,Urgot
456,Lucian
283,Mordekaiser
327,MissFortune
...,...
595,Yasuo
706,Velkoz
741,Lucian


In [42]:
pd.value_counts(na_top3_combined[na_top3_combined['Champion1'] == 'Aatrox'][[f'Champion{i}' for i in range(2, 16)]].values.flatten())

Pantheon    22
Kayn        19
Ezreal      19
Yasuo       18
Lucian      18
            ..
Teemo        1
Katarina     1
Kled         1
Kayle        1
Vi           1
Length: 127, dtype: int64

In [3]:
na_dfs = get_regional_csvs('na')
kr_dfs = get_regional_csvs('kr')
euw_dfs = get_regional_csvs('euw')

In [4]:
na_dfs['diamond'].head()

,Champion1,Mastery1,Champion2,Mastery2,Champion3,Mastery3,Champion4,Mastery4,Champion5,Mastery5
0,875,392982,240,281442,245,271693,14,239201,13,161435
1,92,178830,24,168553,126,119510,164,101045,114,79445
2,55,397277,34,292924,112,96573,121,81492,432,49467
3,412,416106,74,168156,432,132571,117,87439,111,68316
4,18,98100,518,92424,222,70200,89,68836,145,36349


In [5]:
kr_dfs['diamond'].head()

,Champion1,Mastery1,Champion2,Mastery2,Champion3,Mastery3,Champion4,Mastery4,Champion5,Mastery5
0,266,124094,126,77659,39,64466,164,40700,517,39502
1,238,429884,236,134557,145,130125,67,101330,64,85474
2,234,225320,10,179661,777,176145,120,161611,104,119311
3,412,360116,238,212892,92,212037,107,193670,7,182946
4,157,172559,92,118584,91,111408,64,103294,238,101800


In [6]:
euw_dfs['diamond'].head()

,Champion1,Mastery1,Champion2,Mastery2,Champion3,Mastery3,Champion4,Mastery4,Champion5,Mastery5
0,117,145461,267,103061,43,96639,497,85432,235,81444
1,55,2152096,40,434336,37,163254,117,45074,16,30901
2,29,343736,145,227872,81,163469,222,156098,96,102269
3,76,75694,131,75250,104,67350,5,56997,102,44987
4,236,57761,238,34488,8,28604,103,24610,7,24509


In [7]:
na_combined = pd.concat(na_dfs.values(), axis=0)
na_combined.head()

,Champion1,Mastery1,Champion2,Mastery2,Champion3,Mastery3,Champion4,Mastery4,Champion5,Mastery5
0,50,114359,114,92161,427,79025,32,63376,350,57265
1,21,98748,58,65117,13,55159,236,54129,67,53659
2,99,227286,25,90071,61,85030,103,66518,115,65364
3,58,98576,82,44299,3,41376,6,40942,86,39732
4,84,391931,145,139374,67,98345,28,93254,142,85232


In [8]:
na_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7944 entries, 0 to 1836
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Champion1  7944 non-null   int16
 1   Mastery1   7944 non-null   int64
 2   Champion2  7944 non-null   int16
 3   Mastery2   7944 non-null   int64
 4   Champion3  7944 non-null   int16
 5   Mastery3   7944 non-null   int64
 6   Champion4  7944 non-null   int16
 7   Mastery4   7944 non-null   int64
 8   Champion5  7944 non-null   int16
 9   Mastery5   7944 non-null   int64
dtypes: int16(5), int64(5)
memory usage: 450.0 KB


In [17]:
na_combined[['Mastery2', 'Mastery3', 'Mastery4', 'Mastery5']] = na_combined[['Mastery2', 'Mastery3', 'Mastery4', 'Mastery5']].div(na_combined[['Mastery2', 'Mastery3', 'Mastery4', 'Mastery5']].sum(axis=1), axis=0)

In [19]:
# Don't need the first champion's mastery
na_combined = na_combined.drop(['Mastery1'], axis=1)
na_combined.head()

,Champion1,Champion2,Mastery2,Champion3,Mastery3,Champion4,Mastery4,Champion5,Mastery5
0,50,114,0.315807,427,0.270794,32,0.217170,350,0.196229
1,21,58,0.285521,13,0.241858,236,0.237341,67,0.235280
2,99,25,0.293407,61,0.276986,103,0.216683,115,0.212924
3,58,82,0.266302,3,0.248730,6,0.246121,86,0.238847
4,84,145,0.334869,67,0.236290,28,0.224058,142,0.204784


In [24]:
import json

In [10]:
raw_champions = None
with open('data/champions.json', encoding="cp866") as f:
    raw_champions = json.load(f)

In [11]:
raw_champions = None
with open('data/champions.json', encoding="cp866") as f:
    raw_champions = json.load(f)
    
champion_ids_name = {}
for champ in raw_champions['data'].keys():
    champion_ids_name[int(raw_champions['data'][champ]['key'])] = champ

In [34]:
champion_ids_name

{266: 'Aatrox',
 103: 'Ahri',
 84: 'Akali',
 166: 'Akshan',
 12: 'Alistar',
 32: 'Amumu',
 34: 'Anivia',
 1: 'Annie',
 523: 'Aphelios',
 22: 'Ashe',
 136: 'AurelionSol',
 268: 'Azir',
 432: 'Bard',
 53: 'Blitzcrank',
 63: 'Brand',
 201: 'Braum',
 51: 'Caitlyn',
 164: 'Camille',
 69: 'Cassiopeia',
 31: 'Chogath',
 42: 'Corki',
 122: 'Darius',
 131: 'Diana',
 119: 'Draven',
 36: 'DrMundo',
 245: 'Ekko',
 60: 'Elise',
 28: 'Evelynn',
 81: 'Ezreal',
 9: 'Fiddlesticks',
 114: 'Fiora',
 105: 'Fizz',
 3: 'Galio',
 41: 'Gangplank',
 86: 'Garen',
 150: 'Gnar',
 79: 'Gragas',
 104: 'Graves',
 887: 'Gwen',
 120: 'Hecarim',
 74: 'Heimerdinger',
 420: 'Illaoi',
 39: 'Irelia',
 427: 'Ivern',
 40: 'Janna',
 59: 'JarvanIV',
 24: 'Jax',
 126: 'Jayce',
 202: 'Jhin',
 222: 'Jinx',
 145: 'Kaisa',
 429: 'Kalista',
 43: 'Karma',
 30: 'Karthus',
 38: 'Kassadin',
 55: 'Katarina',
 10: 'Kayle',
 141: 'Kayn',
 85: 'Kennen',
 121: 'Khazix',
 203: 'Kindred',
 240: 'Kled',
 96: 'KogMaw',
 7: 'Leblanc',
 64: 'LeeSi

In [35]:
na_combined[[f'Champion{i}' for i in range(1, 6)]] = na_combined[[f'Champion{i}' for i in range(1, 6)]].replace(champion_ids_name)
na_combined.head()

,Champion1,Champion2,Mastery2,Champion3,Mastery3,Champion4,Mastery4,Champion5,Mastery5
0,Swain,Fiora,0.315807,Ivern,0.270794,Amumu,0.217170,Yuumi,0.196229
1,MissFortune,Renekton,0.285521,Ryze,0.241858,Lucian,0.237341,Vayne,0.235280
2,Lux,Morgana,0.293407,Orianna,0.276986,Ahri,0.216683,Ziggs,0.212924
3,Renekton,Mordekaiser,0.266302,Galio,0.248730,Urgot,0.246121,Garen,0.238847
4,Akali,Kaisa,0.334869,Vayne,0.236290,Evelynn,0.224058,Zoe,0.204784


In [54]:
pd.value_counts(na_combined[na_combined.isin(['Ziggs']).any(axis=1)][[f'Champion{i}' for i in range(1, 6)]].values.flatten())

Ziggs           47
Lux              8
Xerath           6
Morgana          6
Diana            5
                ..
Maokai           1
Rakan            1
Fiddlesticks     1
Zoe              1
Sivir            1
Length: 97, dtype: int64

In [40]:
na_combined.groupby('Champion1')[[f'Champion{i}' for i in range(2, 6)]].agg(pd.Series.mode)

,Champion2,Champion3,Champion4,Champion5
Champion1,,,,
Aatrox,"[Darius, Mordekaiser]",Darius,Sett,Camille
Ahri,Lux,Lux,Neeko,Lux
Akali,Yasuo,Yasuo,Ahri,Ezreal
Akshan,"[Katarina, Khazix, Sett, Yorick]","[Kayn, Seraphine, Swain, Viego]","[Graves, Illaoi, Thresh, Zac]","[Camille, Ekko, Morgana, Sylas]"
Alistar,Nautilus,Blitzcrank,Blitzcrank,Rakan
...,...,...,...,...
Zeri,"[Ezreal, Lux]","[Kaisa, MissFortune]","[Teemo, Tristana]","[Caitlyn, Morgana]"
Ziggs,"[Diana, Galio, Neeko, Rumble, Vi]",Akali,"[Corki, Katarina, Malphite, Neeko, Sylas]","[Ahri, Fizz, Lux, Morgana, Xerath]"
Zilean,Thresh,"[Janna, Morgana, Thresh]",Morgana,Janna


In [37]:
pd.value_counts(na_combined[na_combined.isin(['Swain']).any(axis=1)][[f'Champion{i}' for i in range(2, 6)]].values.flatten())

Swain          169
Lux             23
Jhin            19
Xerath          18
Mordekaiser     18
              ... 
Hecarim          1
Xayah            1
Shyvana          1
Elise            1
Quinn            1
Length: 142, dtype: int64

In [81]:
na_combined.value_counts()

Champion1  Champion2  Champion3  Champion4  Champion5
99         267        16         37         40           2
103        84         99         142        147          2
1          3          74         497        267          1
145        84         157        81         39           1
           81         523        236        202          1
                                                        ..
75         157        83         62         38           1
           145        81         235        134          1
           141        45         91         86           1
                      8          121        238          1
888        412        43         432        16           1
Length: 6894, dtype: int64

In [90]:
na_combined.groupby('Champion1').size()

Champion1
1      22
2      11
3      30
4      25
5      17
       ..
777    33
875    88
876     9
887     4
888     1
Length: 159, dtype: int64

In [95]:
champion_ids = sorted(na_combined['Champion1'].unique())
champion_ids[:5]

[1, 2, 3, 4, 5]

In [116]:
na_combined[na_combined.isin([1]).any(axis=1)].head()

,Champion1,Champion2,Champion3,Champion4,Champion5
119,1,99,246,25,518
183,133,101,53,1,6
202,22,63,1,21,145
221,90,3,74,1,86
233,7,18,222,69,1


In [125]:
pd.value_counts(na_combined[na_combined.isin([1]).any(axis=1)].values.flatten())

1      101
99      14
202     13
45      11
245     10
      ... 
15       1
76       1
41       1
56       1
34       1
Length: 124, dtype: int64

In [ ]:
def most_common_ids(id):
    # get all rows with the given id
    return

In [98]:
na_combined.groupby('Champion1')

In [89]:
na_combined.groupby('Champion1')['Champion2'].value_counts().to_frame()

Champion2
Champion1 Champion2           
1         45                 2
          99                 2
          134                2
          3                  1
          8                  1
...                        ...
876       266                1
887       875                2
          38                 1
          518                1
888       412                1

[3948 rows x 1 columns]

In [77]:
na_combined[na_combined['Champion1'] == 1]['Champion2'].value_counts()

99     2
134    2
45     2
18     1
31     1
11     1
77     1
143    1
53     1
58     1
8      1
202    1
117    1
30     1
131    1
157    1
63     1
875    1
3      1
Name: Champion2, dtype: int64

In [78]:
na_combined[na_combined['Champion1'] == 1]['Champion3'].value_counts()

103    2
61     2
22     2
246    1
12     1
32     1
17     1
16     1
98     1
102    1
203    1
64     1
112    1
222    1
90     1
29     1
5      1
245    1
74     1
Name: Champion3, dtype: int64

In [79]:
na_combined[na_combined['Champion1'] == 1]['Champion4'].value_counts()

25     1
29     1
497    1
420    1
103    1
245    1
30     1
58     1
203    1
127    1
201    1
82     1
421    1
98     1
81     1
18     1
63     1
11     1
31     1
120    1
202    1
59     1
Name: Champion4, dtype: int64

In [80]:
na_combined[na_combined['Champion1'] == 1]['Champion5'].value_counts()

45     3
267    2
518    1
13     1
74     1
75     1
28     1
18     1
497    1
117    1
91     1
145    1
22     1
56     1
238    1
50     1
77     1
141    1
134    1
Name: Champion5, dtype: int64

In [68]:
na_combined.pivot_table(index='Champion1', aggfunc=stats.mode)

,Champion2,Champion3,Champion4,Champion5
Champion1,,,,
1,"([45], [2])","([22], [2])","([11], [1])","([45], [3])"
2,"([3], [1])","([141], [2])","([11], [1])","([5], [1])"
3,"([99], [2])","([111], [3])","([104], [2])","([68], [2])"
4,"([35], [2])","([112], [4])","([5], [2])","([22], [2])"
5,"([64], [3])","([9], [2])","([777], [2])","([11], [2])"
...,...,...,...,...
777,"([157], [6])","([36], [2])","([145], [3])","([157], [6])"
875,"([82], [5])","([82], [6])","([157], [4])","([122], [6])"
876,"([81], [2])","([3], [1])","([6], [1])","([51], [1])"


In [55]:
kr_combined = pd.concat(kr_dfs.values(), axis=0)
kr_combined.head()

,Champion1,Mastery1,Champion2,Mastery2,Champion3,Mastery3,Champion4,Mastery4,Champion5,Mastery5
0,157,109353,555,80202,122,53546,81,31706,517,28081
1,266,117626,54,77185,36,60833,53,39345,14,28882
2,58,250388,875,61353,141,53829,516,48399,234,34646
3,157,188270,777,65476,555,49046,517,41462,84,38462
4,122,38797,7,32084,238,25974,875,24392,777,21579


In [81]:
kr_combined[[f'Champion{i}' for i in range(1, 6)]] = kr_combined[[f'Champion{i}' for i in range(1, 6)]].replace(champion_ids_name)

In [82]:
euw_combined = pd.concat(euw_dfs.values(), axis=0)
euw_combined.head()

,Champion1,Mastery1,Champion2,Mastery2,Champion3,Mastery3,Champion4,Mastery4,Champion5,Mastery5
0,86,91814,75,89144,99,77068,32,56638,117,35227
1,157,137297,777,57998,62,27081,141,24226,266,22471
2,222,395187,518,213365,350,127739,498,113112,145,79875
3,98,153711,141,79765,5,75819,19,75211,11,73228
4,143,64077,59,53913,11,46980,412,41843,126,38971


In [83]:
euw_combined[[f'Champion{i}' for i in range(1, 6)]] = euw_combined[[f'Champion{i}' for i in range(1, 6)]].replace(champion_ids_name)

In [84]:
euw_combined[['Mastery2', 'Mastery3', 'Mastery4', 'Mastery5']] = euw_combined[['Mastery2', 'Mastery3', 'Mastery4', 'Mastery5']].div(euw_combined[['Mastery2', 'Mastery3', 'Mastery4', 'Mastery5']].sum(axis=1), axis=0)

In [85]:
kr_combined[['Mastery2', 'Mastery3', 'Mastery4', 'Mastery5']] = kr_combined[['Mastery2', 'Mastery3', 'Mastery4', 'Mastery5']].div(kr_combined[['Mastery2', 'Mastery3', 'Mastery4', 'Mastery5']].sum(axis=1), axis=0)

In [86]:
all_combined = pd.concat([na_combined, euw_combined, kr_combined], axis=0)
all_combined.head()

,Champion1,Champion2,Mastery2,Champion3,Mastery3,Champion4,Mastery4,Champion5,Mastery5,Mastery1
0,Swain,Fiora,0.315807,Ivern,0.270794,Amumu,0.217170,Yuumi,0.196229,NaN
1,MissFortune,Renekton,0.285521,Ryze,0.241858,Lucian,0.237341,Vayne,0.235280,NaN
2,Lux,Morgana,0.293407,Orianna,0.276986,Ahri,0.216683,Ziggs,0.212924,NaN
3,Renekton,Mordekaiser,0.266302,Galio,0.248730,Urgot,0.246121,Garen,0.238847,NaN
4,Akali,Kaisa,0.334869,Vayne,0.236290,Evelynn,0.224058,Zoe,0.204784,NaN


In [87]:
all_combined.shape

(31674, 10)

In [126]:
champ_value_counts = all_combined[all_combined['Champion1'] == 'Aatrox'][['Champion1', 'Champion2', 'Mastery2']][['Champion2']].value_counts().to_frame()
champ_value_counts.head()

,0
Champion2,
Darius,13
Camille,12
Mordekaiser,12
Yasuo,10
Sylas,9


In [133]:
all_combined[all_combined['Champion1'] == 'Aatrox'][['Champion1', 'Champion2', 'Mastery2']].groupby('Champion2')[['Champion2', 'Mastery2']].mean().reset_index()

,Champion2,Mastery2
0,Akali,0.326925
1,Amumu,0.400151
2,Brand,0.362340
3,Camille,0.400544
4,Chogath,0.308308
...,...,...
80,Yorick,0.441589
81,Zac,0.401014
82,Zed,0.337586
83,Zilean,0.297490


In [136]:
testing_df = all_combined[all_combined['Champion1'] == 'Aatrox'][['Champion1', 'Champion2', 'Mastery2']].groupby('Champion2')[['Champion2', 'Mastery2']].mean().reset_index().merge(champ_value_counts, on='Champion2')
testing_df.head()

,Champion2,Mastery2,0
0,Akali,0.326925,3
1,Amumu,0.400151,2
2,Brand,0.362340,1
3,Camille,0.400544,12
4,Chogath,0.308308,1


In [138]:
(testing_df['Mastery2'] * testing_df[0]).sum()

81.85010772614656

In [142]:
testing_df.loc[5]

Champion2      Darius
Mastery2     0.386705
0                  13
Name: 5, dtype: object

In [141]:
((testing_df['Mastery2'] * testing_df[0]) / (testing_df['Mastery2'] * testing_df[0]).sum()).sort_values(ascending=False)

5     0.061419
40    0.060074
3     0.058724
78    0.041676
60    0.036711
        ...   
16    0.003486
61    0.003460
42    0.003449
74    0.003226
20    0.003215
Length: 85, dtype: float64

In [121]:
all_combined[all_combined['Champion1'] == 'Aatrox'][['Champion1', 'Champion2', 'Mastery2']]['Champion2'].value_counts()

Darius         13
Camille        12
Mordekaiser    12
Yasuo          10
Sylas           9
               ..
Illaoi          1
Zilean          1
Trundle         1
Shaco           1
Zac             1
Name: Champion2, Length: 85, dtype: int64

In [112]:
all_combined.groupby(['Champion1', 'Champion2'])[['Champion1', 'Champion2', 'Mastery2']].mean().loc['Aatrox']

,Mastery2
Champion2,
Akali,0.326925
Amumu,0.400151
Brand,0.362340
Camille,0.400544
Chogath,0.308308
...,...
Yorick,0.441589
Zac,0.401014
Zed,0.337586


In [75]:
all_combined.groupby('Champion1')[[f'Champion{i}' for i in range(2, 6)]].agg(pd.Series.mode)['Champion5'].apply(lambda x: isinstance(x, np.ndarray)).sum()

48

In [29]:
pd.value_counts(all_combined[all_combined.isin([1]).any(axis=1)].values.flatten()).index[1:4]

Int64Index([99, 157, 45], dtype='int64')

In [35]:
def top3_by_id(id):
    return np.array(pd.value_counts(all_combined[all_combined.isin([id]).any(axis=1)].values.flatten()).index[1:4], dtype=np.int16)

In [37]:
import json

In [38]:
raw_champions = None
with open('data/champions.json') as f:
    raw_champions = json.load(f)

In [43]:
int(raw_champions['data']['Aatrox']['key'])

'266'

In [45]:
champion_ids = []
for champ in raw_champions['data'].keys():
    champion_ids.append(int(raw_champions['data'][champ]['key']))
champion_ids = np.array(champion_ids, dtype=np.int16)

In [48]:
champion_ids

array([266, 103,  84, 166,  12,  32,  34,   1, 523,  22, 136, 268, 432,
        53,  63, 201,  51, 164,  69,  31,  42, 122, 131, 119,  36, 245,
        60,  28,  81,   9, 114, 105,   3,  41,  86, 150,  79, 104, 887,
       120,  74, 420,  39, 427,  40,  59,  24, 126, 202, 222, 145, 429,
        43,  30,  38,  55,  10, 141,  85, 121, 203, 240,  96,   7,  64,
        89, 876, 127, 236, 117,  99,  54,  90,  57,  11,  21,  62,  82,
        25, 267,  75, 111, 518,  76,  56,  20,   2,  61, 516,  80,  78,
       555, 246, 133, 497,  33, 421, 526, 888,  58, 107,  92,  68,  13,
       360, 113, 235, 147, 875,  35,  98, 102,  27,  14,  15,  72,  37,
        16,  50, 517, 134, 223, 163,  91,  44,  17, 412,  18,  48,  23,
         4,  29,  77,   6, 110,  67,  45, 161, 711, 254, 234, 112,   8,
       106,  19, 498, 101,   5, 157, 777,  83, 350, 154, 238, 221, 115,
        26, 142, 143], dtype=int16)

In [36]:
top3_by_id(2)

array([ 64, 104, 141], dtype=int16)

In [51]:
champion_ids = sorted(champion_ids)

In [52]:
connections = []
for id in champion_ids:
    connections.append(top3_by_id(id))

In [54]:
connections_df = pd.DataFrame(connections, index=champion_ids, columns=['Common1', 'Common2', 'Common3'])
connections_df.head()

,Common1,Common2,Common3
1,99,157,45
2,64,104,141
3,517,99,157
4,81,64,157
5,64,11,141


In [56]:
connections_df.shape

(159, 3)

In [58]:
connections_df.to_csv('out_data/champion_connections.csv')